# Financial Instruments - Complete Homework Solutions
## Bus 35100 - John Heaton
### Assignments 1-7: Comprehensive Analysis

This notebook contains complete solutions to all homework assignments, including:
- HW1: Arbitrage and Forward Rates
- HW2: Exploiting Arbitrage, Commodity Futures, Southwest Hedging
- HW3: Greece Currency Swaps, Options Hedging
- HW4: Barings/Leeson Options, Binomial Trees (FDA)
- HW5: Multi-period Binomial Trees, Black-Scholes
- HW6: Implied Volatility, PLUS Valuation
- HW7: American Options, KMV Model (Citigroup Default)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import norm
from scipy.optimize import fsolve, brentq
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 6)
np.set_printoptions(precision=6, suppress=True)

# Homework 1: Arbitrage and Forward Rates

## Problem 1: Arbitrage and Forward Rates

**Given:**
- Current exchange rate: $M_0 = 1.20$ USD/EUR
- US risk-free rate: $r_{\$} = 5\%$ (continuously compounded)
- EUR risk-free rate: $r_{€} = 4.5\%$ (continuously compounded)
- Maturity: $T = 1$ year

In [ ]:
# HW1 Problem 1: Forward Exchange Rate

# Parameters
M0 = 1.20  # Current exchange rate (USD/EUR)
r_usd = 0.05  # US risk-free rate (continuously compounded)
r_eur = 0.045  # EUR risk-free rate (continuously compounded)
T = 1  # Time to maturity (years)

# Part 1: Calculate theoretical forward rate using no-arbitrage principle
# Formula: F_{0,T} = M_0 * exp((r_$ - r_€) * T)
F_theoretical = M0 * np.exp((r_usd - r_eur) * T)

print("="*60)
print("HOMEWORK 1 - PROBLEM 1: ARBITRAGE AND FORWARD RATES")
print("="*60)
print(f"\nGiven Parameters:")
print(f"  Current exchange rate (M₀): ${M0:.4f} USD/EUR")
print(f"  US risk-free rate (r$): {r_usd*100:.2f}% (continuously compounded)")
print(f"  EUR risk-free rate (r€): {r_eur*100:.2f}% (continuously compounded)")
print(f"  Time to maturity (T): {T} year")
print(f"\n(1) Theoretical Forward Rate (No-Arbitrage):")
print(f"  F₀,T = M₀ × exp((r$ - r€) × T)")
print(f"  F₀,T = {M0:.4f} × exp(({r_usd:.4f} - {r_eur:.4f}) × {T})")
print(f"  F₀,T = {M0:.4f} × exp({r_usd - r_eur:.4f})")
print(f"  F₀,T = {M0:.4f} × {np.exp((r_usd - r_eur) * T):.6f}")
print(f"  F₀,T = ${F_theoretical:.6f} USD/EUR")

# Part 2: Arbitrage opportunity when forward is trading at 1.15
F_market = 1.15
print(f"\n(2) Arbitrage Opportunity Analysis:")
print(f"  Market forward rate: ${F_market:.4f} USD/EUR")
print(f"  Theoretical forward rate: ${F_theoretical:.6f} USD/EUR")
print(f"  Difference: ${F_market - F_theoretical:.6f} USD/EUR")
print(f"\n  Since F_market < F_theoretical, the forward is UNDERPRICED")
print(f"\n  Arbitrage Strategy (for 1 EUR):")
print(f"  " + "="*56)
print(f"  At t=0:")
print(f"    1. Borrow ${M0:.4f} USD at {r_usd*100:.1f}% for 1 year")
print(f"    2. Convert ${M0:.4f} USD to 1 EUR at spot rate")
print(f"    3. Invest 1 EUR at {r_eur*100:.1f}% for 1 year")
print(f"    4. Enter long forward contract to sell EUR at F = ${F_market:.4f}")
print(f"\n  At t=1:")
usd_owe = M0 * np.exp(r_usd * T)
eur_have = 1 * np.exp(r_eur * T)
usd_receive = eur_have * F_market
arbitrage_profit = usd_receive - usd_owe

print(f"    1. EUR investment matures: 1 × e^({r_eur*100:.1f}% × 1) = {eur_have:.6f} EUR")
print(f"    2. Convert {eur_have:.6f} EUR at forward rate ${F_market:.4f}")
print(f"       = ${eur_have * F_market:.6f} USD")
print(f"    3. Repay USD loan: ${M0:.4f} × e^({r_usd*100:.1f}% × 1) = ${usd_owe:.6f} USD")
print(f"\n  Arbitrage Profit: ${arbitrage_profit:.6f} USD per EUR")
print(f"  Profit Percentage: {(arbitrage_profit/M0)*100:.4f}%")
print("="*60)

## Problem 2: Forward Rates and Covered Interest Rate Parity

We will now analyze real market data to test whether Covered Interest Rate Parity holds.

In [ ]:
# HW1 Problem 2: Load and analyze forward rate data

try:
    # Load the data
    df_hw1 = pd.read_excel('Assignments/Assignment 1/DataHW1.xls')
    print("\n" + "="*60)
    print("HOMEWORK 1 - PROBLEM 2: COVERED INTEREST RATE PARITY")
    print("="*60)
    print("\nData loaded successfully!")
    print(f"\nDataset shape: {df_hw1.shape}")
    print(f"\nFirst few rows:")
    display(df_hw1.head())
    
    # Convert LIBOR rates from linear to continuous compounding
    # Formula: r_continuous = ln(1 + r_linear * T/360) / (T/360)
    # For simplicity with LIBOR: r_continuous ≈ ln(1 + r_linear)
    
    maturities = ['1M', '3M', '6M', '1Y']
    days_dict = {'1M': 30, '3M': 90, '6M': 180, '1Y': 360}
    
    results = []
    
    for idx, row in df_hw1.iterrows():
        date = row['Date']
        spot = row['Spot']
        
        for mat in maturities:
            # Get data
            usd_libor = row[f'USD_LIBOR_{mat}'] / 100  # Convert to decimal
            eur_libor = row[f'EUR_LIBOR_{mat}'] / 100  # Convert to decimal
            forward_quoted = row[f'Forward_{mat}']
            
            # Time fraction
            T = days_dict[mat] / 360
            
            # Convert LIBOR (linear compounding) to continuous compounding
            r_usd_cont = np.log(1 + usd_libor * T) / T
            r_eur_cont = np.log(1 + eur_libor * T) / T
            
            # Calculate theoretical forward rate
            forward_theoretical = spot * np.exp((r_usd_cont - r_eur_cont) * T)
            
            # Calculate difference
            difference = forward_quoted - forward_theoretical
            pct_diff = (difference / forward_theoretical) * 100
            
            results.append({
                'Date': date,
                'Maturity': mat,
                'Spot': spot,
                'USD_LIBOR': usd_libor * 100,
                'EUR_LIBOR': eur_libor * 100,
                'Forward_Quoted': forward_quoted,
                'Forward_Theoretical': forward_theoretical,
                'Difference': difference,
                'Pct_Difference': pct_diff
            })
    
    df_results = pd.DataFrame(results)
    
    print("\n(1) Calculated Theoretical Forward Rates:")
    print("\nFormula: F = S₀ × exp((r$ - r€) × T)")
    print("where LIBOR rates are converted from linear to continuous compounding")
    print("\nResults Summary:")
    display(df_results)
    
    print("\n(2) Testing Covered Interest Rate Parity:")
    print("\nAverage Absolute Percentage Differences by Maturity:")
    parity_check = df_results.groupby('Maturity')['Pct_Difference'].agg(['mean', 'std', lambda x: np.abs(x).mean()])
    parity_check.columns = ['Mean (%)', 'Std Dev (%)', 'Avg Abs (%)']
    display(parity_check)
    
    # Check if parity holds (within 0.1%)
    tolerance = 0.1  # 0.1%
    violations = df_results[np.abs(df_results['Pct_Difference']) > tolerance]
    
    if len(violations) > 0:
        print(f"\nParity violations (|difference| > {tolerance}%): {len(violations)} out of {len(df_results)}")
        print("\n(3) Arbitrage Strategy Example:")
        
        # Pick the largest violation
        max_violation = violations.loc[violations['Pct_Difference'].abs().idxmax()]
        print(f"\nLargest Parity Violation:")
        print(f"  Date: {max_violation['Date']}")
        print(f"  Maturity: {max_violation['Maturity']}")
        print(f"  Quoted Forward: ${max_violation['Forward_Quoted']:.6f}")
        print(f"  Theoretical Forward: ${max_violation['Forward_Theoretical']:.6f}")
        print(f"  Difference: {max_violation['Pct_Difference']:.4f}%")
        
        if max_violation['Pct_Difference'] > 0:
            print("\n  Forward is OVERPRICED → Sell forward, create synthetic long")
            print("  Strategy:")
            print("    1. Short the forward contract (sell EUR forward)")
            print("    2. Borrow USD, convert to EUR at spot")
            print("    3. Invest EUR at EUR LIBOR")
            print("    4. At maturity: EUR investment + forward sale - USD loan repayment")
        else:
            print("\n  Forward is UNDERPRICED → Buy forward, create synthetic short")
            print("  Strategy:")
            print("    1. Long the forward contract (buy EUR forward)")
            print("    2. Borrow EUR, convert to USD at spot")
            print("    3. Invest USD at USD LIBOR")
            print("    4. At maturity: USD investment - forward purchase - EUR loan repayment")
    else:
        print(f"\nNo significant parity violations found (all within {tolerance}% tolerance)")
        print("Market appears to be in equilibrium - no arbitrage opportunities.")
    
except FileNotFoundError:
    print("\nData file not found. Creating synthetic example...")
    # Create synthetic data
    dates = pd.date_range('2005-10-01', periods=5, freq='YS')
    np.random.seed(42)
    df_hw1 = pd.DataFrame({
        'Date': dates,
        'Spot': [1.2 + np.random.normal(0, 0.05) for _ in range(5)],
        'USD_LIBOR_1M': [3.5 + np.random.normal(0, 0.3) for _ in range(5)],
        'EUR_LIBOR_1M': [2.0 + np.random.normal(0, 0.3) for _ in range(5)],
        'Forward_1M': [1.21 + np.random.normal(0, 0.02) for _ in range(5)]
    })
    print("Using synthetic data for demonstration...")
    display(df_hw1)

In [ ]:
# Visualization: Forward Rate Parity Analysis

if 'df_results' in locals():
    # Create visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Forward Rates by Maturity', 'Percentage Differences',
                       'Quoted vs Theoretical Forwards', 'Distribution of Differences'),
        specs=[[{'secondary_y': False}, {'secondary_y': False}],
               [{'secondary_y': False}, {'type': 'histogram'}]]
    )
    
    # Plot 1: Forward rates over time
    for mat in maturities:
        data_mat = df_results[df_results['Maturity'] == mat]
        fig.add_trace(
            go.Scatter(x=data_mat['Date'], y=data_mat['Forward_Quoted'],
                      mode='lines+markers', name=f'{mat} Quoted',
                      line=dict(dash='solid')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=data_mat['Date'], y=data_mat['Forward_Theoretical'],
                      mode='lines+markers', name=f'{mat} Theoretical',
                      line=dict(dash='dash')),
            row=1, col=1
        )
    
    # Plot 2: Percentage differences
    for mat in maturities:
        data_mat = df_results[df_results['Maturity'] == mat]
        fig.add_trace(
            go.Scatter(x=data_mat['Date'], y=data_mat['Pct_Difference'],
                      mode='lines+markers', name=mat),
            row=1, col=2
        )
    
    # Plot 3: Scatter plot quoted vs theoretical
    fig.add_trace(
        go.Scatter(x=df_results['Forward_Theoretical'], y=df_results['Forward_Quoted'],
                  mode='markers', name='All Maturities',
                  marker=dict(size=8, color=df_results['Pct_Difference'],
                             colorscale='RdBu', showscale=True,
                             colorbar=dict(title='% Diff', x=0.46, y=0.25, len=0.4))),
        row=2, col=1
    )
    # Add 45-degree line
    min_val = min(df_results['Forward_Theoretical'].min(), df_results['Forward_Quoted'].min())
    max_val = max(df_results['Forward_Theoretical'].max(), df_results['Forward_Quoted'].max())
    fig.add_trace(
        go.Scatter(x=[min_val, max_val], y=[min_val, max_val],
                  mode='lines', name='Parity Line',
                  line=dict(dash='dash', color='black')),
        row=2, col=1
    )
    
    # Plot 4: Histogram of differences
    fig.add_trace(
        go.Histogram(x=df_results['Pct_Difference'], nbinsx=30,
                    name='Distribution', marker=dict(color='lightblue', 
                    line=dict(color='darkblue', width=1))),
        row=2, col=2
    )
    
    # Update layout
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=2)
    fig.update_xaxes(title_text="Theoretical Forward", row=2, col=1)
    fig.update_xaxes(title_text="% Difference", row=2, col=2)
    
    fig.update_yaxes(title_text="Forward Rate (USD/EUR)", row=1, col=1)
    fig.update_yaxes(title_text="% Difference", row=1, col=2)
    fig.update_yaxes(title_text="Quoted Forward", row=2, col=1)
    fig.update_yaxes(title_text="Frequency", row=2, col=2)
    
    fig.update_layout(height=800, showlegend=True,
                     title_text="HW1: Covered Interest Rate Parity Analysis")
    
    fig.show()
    
    print("\nVisualization complete!")

# Homework 2: Exploiting Arbitrage, Commodity Futures, and Hedging

## Problem 1: Exploiting an Apparent Arbitrage Opportunity

In [ ]:
# HW2 Problem 1: Arbitrage Trade on Oct 1, 2008

print("\n" + "="*60)
print("HOMEWORK 2 - PROBLEM 1: EXPLOITING ARBITRAGE OPPORTUNITY")
print("="*60)

# Load HW2 data
try:
    df_hw2 = pd.read_excel('Assignments/Assignment 2/DataHW2_2024.xls', sheet_name=None)
    print("\nData sheets available:")
    for sheet_name in df_hw2.keys():
        print(f"  - {sheet_name}")
        print(f"    Shape: {df_hw2[sheet_name].shape}")
        display(df_hw2[sheet_name].head(3))
        
except FileNotFoundError:
    print("\nData file not found. Creating synthetic data for demonstration...")
    # This will be filled in based on actual data structure

## Problem 2: Commodity Futures

We analyze whether the no-arbitrage relation $F_{0,T} = S_t e^{(r+u)T}$ holds for oil futures.

In [ ]:
# HW2 Problem 2: Commodity Futures No-Arbitrage Analysis

print("\n" + "="*60)
print("HOMEWORK 2 - PROBLEM 2: COMMODITY FUTURES")
print("="*60)

print("\nAnalyzing no-arbitrage relation: F₀,T = Sₜ × e^((r+u)×T)")
print("where:")
print("  F₀,T = Futures price at time 0 for delivery at T")
print("  Sₜ = Spot oil price")
print("  r = continuously compounded interest rate")
print("  u = storage cost (as % of oil price)")
print("  T = time to maturity")

print("\n" + "-"*60)
print("Case 1: F₀,T < Sₜ × e^((r+u)×T) - Forward is UNDERPRICED")
print("-"*60)
print("\nArbitrage Strategy (Cash-and-Carry):")
print("  At t=0:")
print("    1. Borrow Sₜ dollars at rate r")
print("    2. Buy 1 unit of oil in spot market for Sₜ")
print("    3. Pay storage costs u×Sₜ×dt continuously")
print("    4. Enter LONG forward contract at F₀,T")
print("\n  At t=T:")
print("    1. Deliver oil via forward contract, receive F₀,T")
print("    2. Repay loan: Sₜ × e^(r×T)")
print("    3. Total storage costs paid: Sₜ × e^(u×T) - Sₜ (approximately u×Sₜ×T)")
print("\n  Profit = F₀,T - Sₜ × e^((r+u)×T) > 0 ✓ ARBITRAGE!")

print("\n" + "-"*60)
print("Case 2: F₀,T > Sₜ × e^((r+u)×T) - Forward is OVERPRICED")
print("-"*60)
print("\nReverse Cash-and-Carry:")
print("  At t=0:")
print("    1. Short sell 1 unit of oil, receive Sₜ")
print("    2. Invest Sₜ at rate r")
print("    3. Save storage costs u×Sₜ×dt")
print("    4. Enter SHORT forward contract at F₀,T")
print("\n  At t=T:")
print("    1. Buy oil via forward contract, pay F₀,T")
print("    2. Receive from investment: Sₜ × e^(r×T)")
print("    3. Saved storage costs: Sₜ × (e^(u×T) - 1)")
print("    4. Return oil to cover short")
print("\n  Profit = Sₜ × e^((r+u)×T) - F₀,T > 0 ✓ ARBITRAGE!")

print("\n" + "-"*60)
print("IMPORTANT CONSTRAINT: Reverse Cash-and-Carry")
print("-"*60)
print("\nThe reverse cash-and-carry may NOT be feasible for commodities because:")
print("  1. Short selling physical oil is difficult/impossible")
print("  2. Cannot 'save' storage costs without owning oil")
print("  3. Convenience yield - oil owners value having physical inventory")
print("\nTherefore: The no-arbitrage relation gives an UPPER BOUND:")
print("  F₀,T ≤ Sₜ × e^((r+u)×T)")
print("\nBut F₀,T can be LESS than this bound without creating arbitrage!")
print("The difference is called the 'convenience yield' (y):")
print("  F₀,T = Sₜ × e^((r+u-y)×T)")
print("\nConvenience yield represents the benefit of holding physical inventory.")
print("="*60)

## Problem 3: Southwest Jet Fuel Hedging

We analyze Southwest Airlines' hedging strategy using crude oil futures.

In [ ]:
# HW2 Problem 3: Southwest Jet Fuel Hedging

print("\n" + "="*60)
print("HOMEWORK 2 - PROBLEM 3: SOUTHWEST JET FUEL HEDGING")
print("="*60)

# Given parameters
annual_fuel_consumption = 1511e6  # gallons per year
jet_fuel_price_dec31_2007 = 2.71  # USD per gallon
hedge_ratio = 0.75  # 75% of Q1 2008 fuel consumption
gallons_per_barrel = 42  # gallons in one barrel
contract_size = 42000  # gallons per contract (1000 barrels)

# Q1 consumption (3 months = 1/4 of annual)
q1_consumption = annual_fuel_consumption / 4
hedged_consumption = q1_consumption * hedge_ratio

print(f"\nGiven Information (Dec 31, 2007):")
print(f"  Annual fuel consumption: {annual_fuel_consumption/1e6:,.0f} million gallons")
print(f"  Q1 2008 consumption: {q1_consumption/1e6:,.2f} million gallons")
print(f"  Hedge ratio: {hedge_ratio*100:.0f}%")
print(f"  Hedged quantity: {hedged_consumption/1e6:,.2f} million gallons")
print(f"  Jet fuel spot price: ${jet_fuel_price_dec31_2007:.2f}/gallon")
print(f"\nContract specifications:")
print(f"  Contract size: {contract_size:,} gallons ({contract_size/gallons_per_barrel:,.0f} barrels)")

# Crude oil futures prices (Dec 31, 2007)
futures_prices = {
    'FEB.08': 95.98,
    'MAR.08': 95.78,
    'APR.08': 95.24
}

print(f"\nCrude Oil Futures Prices ($/barrel):")
for contract, price in futures_prices.items():
    print(f"  {contract}: ${price:.2f}")

# Part 1: Determine hedging strategy
print("\n" + "-"*60)
print("Part 1: Hedging Strategy")
print("-"*60)

# Monthly consumption (assuming uniform consumption)
monthly_consumption = hedged_consumption / 3
contracts_per_month = monthly_consumption / contract_size

print(f"\nMonthly hedged consumption: {monthly_consumption/1e6:,.3f} million gallons")
print(f"Number of contracts per month: {contracts_per_month:,.2f}")
print(f"Rounded contracts per month: {int(np.round(contracts_per_month)):,}")

# Hedging strategy
hedge_strategy = {
    'FEB.08': int(np.round(contracts_per_month)),
    'MAR.08': int(np.round(contracts_per_month)),
    'APR.08': int(np.round(contracts_per_month))
}

print("\nHedging Strategy:")
print("  Action: LONG (buy) crude oil futures contracts")
print("  Rationale: Southwest needs to buy jet fuel, so we hedge by")
print("             taking long positions in correlated crude oil futures")
print("\nPosition Details:")
total_contracts = 0
total_notional = 0
for contract, num_contracts in hedge_strategy.items():
    notional = num_contracts * (futures_prices[contract] * 1000)  # 1000 barrels per contract
    total_contracts += num_contracts
    total_notional += notional
    print(f"  {contract}: {num_contracts:,} contracts (Notional: ${notional/1e6:,.2f}M @ ${futures_prices[contract]:.2f}/bbl)")

print(f"\n  Total contracts: {total_contracts:,}")
print(f"  Total notional value: ${total_notional/1e6:,.2f} million")
print(f"  Total gallons hedged: {total_contracts * contract_size/1e6:,.2f} million gallons")

print("\n" + "="*60)

In [ ]:
# HW2 Problem 3 Part 2: P&L Analysis (need actual price data)

print("\n" + "-"*60)
print("Part 2: P&L Analysis and Hedging Effectiveness")
print("-"*60)

# This section would analyze actual futures prices from the data file
# showing how the hedge performed and calculating implicit fuel prices

print("\nNote: Complete P&L analysis requires futures price time series data")
print("from DataHW2_2024.xls showing FEB.08, MAR.08, APR.08 contract prices")
print("over time and jet fuel spot prices during Q1 2008.")

print("\nKey metrics to calculate:")
print("  1. Futures P&L by contract expiry")
print("  2. Actual jet fuel costs (unhedged)")
print("  3. Effective jet fuel price (hedged) = Actual fuel cost - Futures P&L")
print("  4. Correlation between jet fuel prices and crude oil futures")
print("  5. Hedge effectiveness ratio")

# Homework 3: Greece Currency Swaps and Options Hedging

## Problem 1: Greece Currency Swaps

In [ ]:
# HW3 Problem 1: Greece Currency Swap

print("\n" + "="*60)
print("HOMEWORK 3 - PROBLEM 1: GREECE CURRENCY SWAPS")
print("="*60)

# Given parameters
N_usd = 50e9  # USD 50 billion
N_eur = 59e9  # EUR 59 billion
spot_rate = N_usd / N_eur  # 0.8475 USD/EUR
coupon_rate_usd = 0.06  # 6% annual, paid semiannually
T = 10  # years
freq = 2  # semiannual

print(f"\nBond Details:")
print(f"  Face Value: ${N_usd/1e9:.0f} billion USD")
print(f"  Coupon Rate: {coupon_rate_usd*100:.0f}% per annum (semiannual)")
print(f"  Maturity: {T} years")
print(f"  Semiannual coupon payment: ${(coupon_rate_usd/freq) * N_usd/1e9:.1f} billion USD")

print(f"\nSwap Details:")
print(f"  Initial exchange: Greece pays ${N_usd/1e9:.0f}B USD, receives €{N_eur/1e9:.0f}B EUR")
print(f"  Exchange rate: ${spot_rate:.4f} USD/EUR")
print(f"  Every 6 months: Greece receives ${(coupon_rate_usd/freq) * N_usd/1e9:.1f}B USD")
print(f"                  Greece pays ??? EUR (to be determined)")
print(f"  At maturity: Greece pays €{N_eur/1e9:.0f}B EUR, receives ${N_usd/1e9:.0f}B USD")

# Load zero coupon bond prices
try:
    df_zcb = pd.read_excel('Assignments/Assignment 3/Greece_GS_table1.xls')
    print(f"\nZero Coupon Bond Prices loaded:")
    display(df_zcb.head(10))
    
    # Extract ZCB prices
    Z_usd = df_zcb['US ZCB'].values
    Z_eur = df_zcb['Greek ZCB'].values
    maturities = df_zcb['Maturity'].values
    
except:
    print("\nUsing ZCB prices from problem statement:")
    # Create DataFrame from problem data
    zcb_data = [
        (0.0, 1.0000, 1.0000), (0.5, 0.9786, 0.9822), (1.0, 0.9588, 0.9647),
        (1.5, 0.9388, 0.9431), (2.0, 0.9191, 0.9192), (2.5, 0.8989, 0.8973),
        (3.0, 0.8788, 0.8749), (3.5, 0.8583, 0.8520), (4.0, 0.8379, 0.8287),
        (4.5, 0.8177, 0.8051), (5.0, 0.7977, 0.7812), (5.5, 0.7780, 0.7603),
        (6.0, 0.7583, 0.7397), (6.5, 0.7370, 0.7194), (7.0, 0.7155, 0.6993),
        (7.5, 0.6953, 0.6795), (8.0, 0.6751, 0.6600), (8.5, 0.6559, 0.6407),
        (9.0, 0.6369, 0.6218), (9.5, 0.6208, 0.6032), (10.0, 0.6050, 0.5848)
    ]
    df_zcb = pd.DataFrame(zcb_data, columns=['Maturity', 'Greek ZCB', 'US ZCB'])
    display(df_zcb)
    
    Z_eur = df_zcb['Greek ZCB'].values
    Z_usd = df_zcb['US ZCB'].values
    maturities = df_zcb['Maturity'].values

# Calculate swap rate
print("\n" + "-"*60)
print("Part 1: Calculate Fair Swap Rate")
print("-"*60)

# Value of USD leg (Greece receives)
usd_coupon = (coupon_rate_usd / freq) * N_usd
usd_cashflows = []
zcb_times = []
for i, t in enumerate(maturities):
    if t > 0 and t <= T:
        zcb_times.append(t)
        if t < T:
            usd_cashflows.append(usd_coupon * Z_usd[i])
        else:  # Maturity
            usd_cashflows.append((usd_coupon + N_usd) * Z_usd[i])

value_usd_leg = sum(usd_cashflows)

print(f"\nUSD Leg (Greece receives):")
print(f"  Semiannual coupon: ${usd_coupon/1e9:.3f}B USD")
print(f"  Present value of coupons + principal: ${value_usd_leg/1e9:.4f}B USD")

# Value of EUR leg (Greece pays)
# At t=0: Greece receives EUR principal
# At t=0.5, 1.0, ..., T: Greece pays EUR coupon
# At t=T: Greece pays EUR principal

# For zero value swap: PV(EUR leg) = PV(USD leg) in EUR terms
# Convert USD leg value to EUR
value_usd_leg_in_eur = value_usd_leg / spot_rate

# EUR leg value = Principal received now - PV(coupons) - PV(principal payment)
# We want: N_eur - Σ(c_eur × N_eur × Z_eur(t)) - N_eur × Z_eur(T) = value_usd_leg_in_eur
# Solving for c_eur (annualized swap rate)

# Sum of ZCB prices for coupon times
sum_zcb_eur = 0
for i, t in enumerate(maturities):
    if 0 < t <= T:
        sum_zcb_eur += Z_eur[i]

# At initiation, value = 0:
# 0 = -value_usd_leg_in_eur + N_eur - (c_eur/2 × N_eur × sum_zcb_eur) - N_eur × Z_eur[-1]
# Solving for c_eur:
# (c_eur/2) × N_eur × sum_zcb_eur = N_eur - N_eur × Z_eur[-1] - value_usd_leg_in_eur

idx_maturity = np.where(maturities == T)[0][0]
Z_eur_T = Z_eur[idx_maturity]

# Rearranging:
# c_eur = 2 × (N_eur × (1 - Z_eur_T) - value_usd_leg_in_eur) / (N_eur × sum_zcb_eur)
c_eur = 2 * (N_eur * (1 - Z_eur_T) - value_usd_leg_in_eur) / (N_eur * sum_zcb_eur)

print(f"\nEUR Leg (Greece pays):")
print(f"  Sum of ZCB prices: {sum_zcb_eur:.6f}")
print(f"  ZCB price at T={T}: {Z_eur_T:.6f}")
print(f"  Fair swap rate (c_eur): {c_eur*100:.4f}% per annum")
print(f"  Semiannual EUR payment: €{(c_eur/2) * N_eur/1e9:.4f}B")

# Verify: Value of swap should be zero
eur_coupons_pv = (c_eur / 2) * N_eur * sum_zcb_eur
eur_principal_pv = N_eur * Z_eur_T
value_eur_leg = N_eur - eur_coupons_pv - eur_principal_pv

print(f"\nVerification:")
print(f"  PV of USD leg (in EUR): €{value_usd_leg_in_eur/1e9:.4f}B")
print(f"  Value of EUR leg: €{value_eur_leg/1e9:.4f}B")
print(f"  Difference: €{(value_eur_leg - value_usd_leg_in_eur)/1e9:.6f}B ≈ 0 ✓")

print("\n" + "="*60)

In [ ]:
# HW3 Problem 1 Part 2: Goldman Sachs Swap

print("\n" + "-"*60)
print("Part 2: Goldman Sachs Swap Analysis")
print("-"*60)

# Goldman Sachs swap parameters
spot_rate_gs = 0.8148  # Historical average USD/EUR (March 12 - June 1, 2001)
c_eur_gs = 0.07  # 7% swap rate quoted by Goldman Sachs
N_eur_gs = N_usd / spot_rate_gs  # EUR principal using GS exchange rate

print(f"\nGoldman Sachs Swap Terms:")
print(f"  Exchange rate used: ${spot_rate_gs:.4f} USD/EUR (vs market ${spot_rate:.4f})")
print(f"  EUR principal: €{N_eur_gs/1e9:.4f}B (vs VeroTende €{N_eur/1e9:.0f}B)")
print(f"  Swap rate: {c_eur_gs*100:.2f}% (vs VeroTende {c_eur*100:.4f}%)")

# Calculate value of GS swap at inception
# USD leg value (same as before)
value_usd_leg_gs_in_eur = value_usd_leg / spot_rate  # Using MARKET spot rate

# EUR leg for GS swap
eur_coupons_pv_gs = (c_eur_gs / 2) * N_eur_gs * sum_zcb_eur
eur_principal_pv_gs = N_eur_gs * Z_eur_T
value_eur_leg_gs = N_eur_gs - eur_coupons_pv_gs - eur_principal_pv_gs

# Value of swap to Greece (in EUR)
swap_value_to_greece = value_eur_leg_gs - value_usd_leg_gs_in_eur

print(f"\nGoldman Sachs Swap Valuation:")
print(f"  Value of EUR leg (Greece pays): €{value_eur_leg_gs/1e9:.4f}B")
print(f"  Value of USD leg (Greece receives, in EUR): €{value_usd_leg_gs_in_eur/1e9:.4f}B")
print(f"  Net value to Greece: €{swap_value_to_greece/1e9:.4f}B")
print(f"  Net value to Greece (USD): ${swap_value_to_greece * spot_rate/1e9:.4f}B")

if swap_value_to_greece > 0:
    print(f"\n  ✓ Positive value! Greece receives upfront benefit of €{swap_value_to_greece/1e9:.4f}B")
else:
    print(f"\n  ✗ Negative value! Greece pays upfront cost of €{-swap_value_to_greece/1e9:.4f}B")

print(f"\nWhy Greece Accepted Goldman's Proposal:")
print(f"  1. UPFRONT CASH: Using off-market exchange rate (${spot_rate_gs:.4f} vs ${spot_rate:.4f})")
print(f"     Greece gets €{N_eur_gs/1e9:.4f}B instead of €{N_eur/1e9:.0f}B initially")
print(f"     Extra EUR received: €{(N_eur_gs - N_eur)/1e9:.4f}B")
print(f"  2. ACCOUNTING BENEFIT: Extra EUR can be counted as revenue initially")
print(f"  3. HIDDEN COST: Higher swap rate ({c_eur_gs*100:.0f}% vs {c_eur*100:.2f}%) means")
print(f"     higher future payments, but these are spread over {T} years")
print(f"  4. OFF-BALANCE SHEET: The swap structure may have received favorable")
print(f"     accounting treatment under rules at the time")

# Cash flow analysis
print(f"\nCash Flow Comparison:")
print(f"\n  Time t=0:")
print(f"    VeroTende: Greece receives €{N_eur/1e9:.0f}B")
print(f"    Goldman:   Greece receives €{N_eur_gs/1e9:.4f}B")
print(f"    Difference: €{(N_eur_gs - N_eur)/1e9:.4f}B more with Goldman ✓")

print(f"\n  Every 6 months (for {T} years = {T*2} payments):")
verotende_payment = (c_eur / 2) * N_eur
goldman_payment = (c_eur_gs / 2) * N_eur_gs
print(f"    VeroTende: Greece pays €{verotende_payment/1e9:.4f}B")
print(f"    Goldman:   Greece pays €{goldman_payment/1e9:.4f}B")
print(f"    Difference: €{(goldman_payment - verotende_payment)/1e9:.4f}B more with Goldman ✗")

print(f"\n  At maturity (t={T}):")
print(f"    VeroTende: Greece pays €{N_eur/1e9:.0f}B principal")
print(f"    Goldman:   Greece pays €{N_eur_gs/1e9:.4f}B principal")
print(f"    Difference: €{(N_eur_gs - N_eur)/1e9:.4f}B more with Goldman ✗")

print(f"\n  TIMING IS KEY:")
print(f"    - Greece gets cash NOW (t=0)")
print(f"    - Greece pays extra over time (spread over {T} years)")
print(f"    - Politicians value present cash over future obligations")
print(f"    - Helps meet budget targets in current year")

print("\n" + "="*60)

Due to the length and complexity of this task, I'll create a comprehensive notebook with all 7 homeworks. However, this response is getting very long. Let me continue with creating a more complete version and save it to the file, then create the marimo version and markdown document.

Let me continue building out the complete solution...